# Trabalho 01 - Experimento SSVEP offline

Dennis Felipe Urtubia e Pedro Perozin

# Carregamento dos dados, labels e descritor

In [1]:
import numpy as np
import json
import mne
import scipy
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from scipy.signal import stft
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

data = np.load('files/data.npy')
labels = np.load('files/labels.npy')

desc_file = open('files/descriptor.json')
descriptor = json.loads(desc_file.read())
desc_file.close()

print('Estruturas => dados', data.shape, 'labels', labels.shape)
print(labels)

Estruturas => dados (125, 257, 1205) labels (125,)
[4 2 3 5 1 2 5 4 2 3 1 5 4 3 2 4 1 2 5 3 4 1 3 1 3 4 2 3 5 1 2 5 4 2 3 1 5
 4 3 2 4 1 2 5 3 4 1 3 1 3 4 2 3 5 1 2 5 4 2 3 1 5 4 3 2 4 1 2 5 3 4 1 3 1
 3 4 2 3 5 1 2 5 4 2 3 1 5 4 3 2 4 1 2 5 3 4 1 3 1 3 4 2 3 5 1 2 5 4 2 3 1
 5 4 3 2 4 1 2 5 3 4 1 3 1 3]


# Criação do EpochsArray

In [2]:
data = data[:,:256,:]

trial_duration = 5
sampling_frequency = data.shape[-1] / trial_duration
montage = mne.channels.make_standard_montage('EGI_256')
ch_names = data.shape[1]
ch_types = 'eeg'

# primeiramente devemos criar o objeto info
info = mne.create_info(montage.ch_names, sampling_frequency, ch_types)

# definindo a montagem do experimento
info.set_montage(montage)

# por fim a criação do EpochsArray
events = np.array([[index, 0, event] for index, event in enumerate(labels)])
# objeto MNE epoch
epoch = mne.EpochsArray(data, info, events)

125 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped


# Aplicando filtros espaciais

In [3]:
channels = ['E108', 'E109', 'E116', 'E125', 'E118', 'E117', 'E126', 'E139', 'E127', 'E138', 'E140', 'E150', 'E151']

filtered_epoch = epoch.copy().pick_channels(channels)
filtered_epoch.filter(l_freq = 5.0, h_freq = 14.0)

# CAR
mne.set_eeg_reference(filtered_epoch, ref_channels=channels)

Setting up band-pass filter from 5 - 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 5.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.00 Hz)
- Upper passband edge: 14.00 Hz
- Upper transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 15.75 Hz)
- Filter length: 399 samples (1.656 sec)

EEG channel type selected for re-referencing
Applying a custom EEG reference.


(<EpochsArray  |   125 events (all good), 0 - 4.99585 sec, baseline off, ~15.1 MB, data loaded,
  '1': 25
  '2': 25
  '3': 30
  '4': 25
  '5': 20>,
 array([[ 0.10401822,  3.14769797,  6.23945143, ...,  3.9958133 ,
          4.24136166,  4.23639578],
        [ 4.41104306,  2.4045516 ,  0.25018658, ...,  6.62193568,
          4.19169337,  1.76657325],
        [ 1.08328458,  2.15904374,  3.04747703, ..., -2.55647088,
         -1.00350958,  0.60783976],
        ...,
        [-0.19247003, -0.12680587,  0.02390848, ..., -1.64050085,
         -1.68820552, -1.44645465],
        [-0.89572343,  1.08075274,  3.24166156, ..., -4.26297305,
         -3.13652035, -1.92687402],
        [-1.21096887, -0.9193238 , -0.47196493, ..., -4.40661384,
         -2.99276488, -1.55852034]]))

# Extração de características

O método `psd_multitaper` nos retorna as características já no domínio da frequência, sem precisar passar por o método stft, por exemplo.

In [4]:
X, _ = mne.time_frequency.psd_multitaper(filtered_epoch)
print('Shape dos dados:', X.shape)

    Using multitaper spectrum estimation with 7 DPSS windows
Shape dos dados: (125, 13, 603)


## Finalizando o vetor de características

In [5]:
X = X.reshape(X.shape[0], X.shape[1] * X.shape[2])

print('Shape dos dados:', X.shape)

Shape dos dados: (125, 7839)


## Adaptação do vetor de labels

In [6]:
# y = np.load('files/labels.npy')
# print('Shape original dos labels', y.shape)

# size = int(X.shape[0] / y.shape[0])
# y = np.concatenate([y for i in range(size)])
# print('Shape final dos labels', y.shape)

# Classificação

### SVM - Support Vector Machine

Foram dispostos alguns laços de repetições aninhados para testar o classificador com diversos parâmetros.
É disposto apenas resultados com acurácia "interessantes" para o experimento.

In [ ]:
for count in range(20):
    for kernel in ['linear', 'poly', 'rbf', 'sigmoid']:
        for gamma in [10, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001]:
            for C in [0.01, 0.1, 1, 10, 100, 1000]:
                X_train, X_test, y_train, y_test = train_test_split(X, labels, train_size=0.7, shuffle=True)
                clf = SVC(gamma=gamma, kernel=kernel, C=C)
                clf.fit(X_train, y_train)
                res = clf.predict(X_test)
                tot_hit = sum([1 for i in range(len(res)) if res[i] == y_test[i]])
                accuracy = tot_hit / X_test.shape[0] * 100
                if accuracy >= 50:
                    print('Kernel:{} | Gamma:{} e C:{} | Accuracy: {:.2f}%'.format(kernel, gamma, C, tot_hit / X_test.shape[0] * 100))

print('Finish SVM')

Kernel:linear | Gamma:10 e C:0.01 | Accuracy: 26.32%
Kernel:linear | Gamma:10 e C:0.1 | Accuracy: 28.95%
Kernel:linear | Gamma:10 e C:1 | Accuracy: 42.11%
Kernel:linear | Gamma:10 e C:10 | Accuracy: 18.42%
Kernel:linear | Gamma:10 e C:100 | Accuracy: 34.21%
Kernel:linear | Gamma:10 e C:1000 | Accuracy: 28.95%
Kernel:linear | Gamma:1 e C:0.01 | Accuracy: 34.21%
Kernel:linear | Gamma:1 e C:0.1 | Accuracy: 28.95%
Kernel:linear | Gamma:1 e C:1 | Accuracy: 28.95%
Kernel:linear | Gamma:1 e C:10 | Accuracy: 39.47%
Kernel:linear | Gamma:1 e C:100 | Accuracy: 23.68%
Kernel:linear | Gamma:1 e C:1000 | Accuracy: 36.84%
Kernel:linear | Gamma:0.1 e C:0.01 | Accuracy: 44.74%
Kernel:linear | Gamma:0.1 e C:0.1 | Accuracy: 34.21%
Kernel:linear | Gamma:0.1 e C:1 | Accuracy: 34.21%
Kernel:linear | Gamma:0.1 e C:10 | Accuracy: 34.21%
Kernel:linear | Gamma:0.1 e C:100 | Accuracy: 39.47%
Kernel:linear | Gamma:0.1 e C:1000 | Accuracy: 36.84%
Kernel:linear | Gamma:0.01 e C:0.01 | Accuracy: 39.47%
Kernel:linea